In [1]:
from collections import Counter
import pandas as pd
import re
from import_utils import utils

In [2]:
file = "entities/data/Inventare_st_michaelsburg_gesamt_korr.xlsx"

In [3]:
dfs = utils.get_dfs(file, filterstring='V')
len(dfs)

51

In [4]:
# import easy values
# create baisc vocabs and relations
ww_kind, _ = WorkWorkRelation.objects.get_or_create(
    name='ist Teil von'
)
ww_kind.name_reverse = 'beinhaltet'
ww_kind.save()
sla, _ = Institution.objects.get_or_create(
    name='Südtiroler Landesarchiv'
)
sla_vb_kind, _ = InstitutionWorkRelation.objects.get_or_create(
    name = 'archiviert'
)
sla_vb_kind.name_reverse = 'ist archiviert in'
sla_vb_kind.save()
for df in dfs:
    vb_name = df.loc[0][0].split(',')[0]
    vb_date = "{}".format([int(s) for s in vb_name.split() if s.isdigit()][0])
    vb, _ = Work.objects.get_or_create(
        name=vb_name
    )
    vb_kind, _ = WorkType.objects.get_or_create(
            name='Verfachbuch'
        )
    vb.kind = vb_kind
    vb.start_date_written = vb_date
    vb.end_date_written = "{}-12-31".format(vb_date)
    vb.save()
    sla_vb, _ = InstitutionWork.objects.get_or_create(
        relation_type=sla_vb_kind,
        related_institution=sla,
        related_work=vb
    )
    for i, row in df.iterrows():
        temp_work, _ = Work.objects.get_or_create(
            written_name=row['Signatur']
        )
        temp_work.name = row['Signatur']
        
    # add start and end date
        if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
            pass
        else:
            date_string_list = row['Datum der Amtshandlung'].split('\n')
            dates = utils.get_start_end(date_string_list)
            if dates:
                if len(dates) > 1:
                    temp_work.start_date_written, temp_work.end_date_written = dates
                    temp_work.save()
                else:
                    temp_work.start_date.written = dates[0]
                    temp_work.save()
        
        if row['Buch/Bücher'] == 'FALSE' or row['Buch/Bücher'] == 'n. a.':
            temp_work.buecher = ""
        else:
            temp_work.buecher = row['Buch/Bücher']
        if row['Summe Inventar'] == 'FALSE' or row['Summe Inventar'] == 'n. a.':
            temp_work.summe_inventar = ""
        else:
            temp_work.summe_inventar = row['Summe Inventar']
        if row['Barschaft (teilweise eigene Berechnung)'] == 'FALSE' or row['Barschaft (teilweise eigene Berechnung)'] == 'n.a.':
            temp_work.barschaft= ""
        else:
            temp_work.barschaft = row['Barschaft (teilweise eigene Berechnung)']
        temp_work.save()
    # kind
        raw_kind = row['Dokumententypus'].split('\n')[0].split(';')[0]
        work_kind, _ = WorkType.objects.get_or_create(
            name=raw_kind
        )
        temp_work.kind = work_kind
        temp_work.save()
    # link to vb
        temp_work_ww, _ = WorkWork.objects.get_or_create(
            relation_type=ww_kind,
            related_workA=temp_work,
            related_workB=vb
        )

    # add start and end date
        if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
            pass
        else:
            date_string_list = row['Datum der Amtshandlung'].split('\n')
            dates = utils.get_start_end(date_string_list)
            if dates:
                if len(dates) > 1:
                    temp_work_ww.start_date_written, temp_work_ww.end_date_written = dates
                    temp_work_ww.save()
                else:
                    temp_work_ww.start_date.written = dates[0]
                    temp_work_ww.save()
            else:
                pass

In [5]:
# add info about money, books, ...
temp_work = None
for df in dfs:
    for i, row in df.iterrows():
        temp_work = Work.objects.get(written_name=row['Signatur'])
        if row['Summe Inventar'] == 'FALSE' or row['Summe Inventar'] == 'n. a.':
            pass
        else:
            moneylist = [int(s) for s in row['Summe Inventar'].split() if s.isdigit()]
            if moneylist:
                try:
                    f, xr = moneylist[0], moneylist[1]
                except IndexError:
                    f, xr = moneylist[0], 0
            temp_work.invenatar_summe_norm = float("{}.{}".format(f, xr))
        temp_work.save()

In [6]:
# Hauptpersonen, deren Wohnorte und jobs
temp_work = None
place = None
parent_pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name='geführt als Hauptperson', name_reverse='führt als Hauptperson'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
pers_place, _ = PersonPlaceRelation.objects.get_or_create(
    name='lebte in', name_reverse='Wohnort von'
)
for df in dfs:
    for i, row in df.iterrows():
        place = utils.create_places_from_place_string(
            row['Wohnort; Ort des betreffenden Gutes/der betreffenden Behausung'].split('\n')[0]
        )
        all_names = row['Name\n(Erklärung aus Verfachbuch)'].split('\n')[0]
        job = row['Beruf'].strip().split('\n')[0]
        if job.startswith('(') or job.startswith('n.') or ":" in job or ";" in job:
            job_obj = None
        else:
            job = job.replace('"', "")
            job_obj, _ = ProfessionType.objects.get_or_create(
                name=job
            )
        for written_name in all_names.split(';'):
            written_name = written_name.strip()
            pers = None
            pers, _ = Person.objects.get_or_create(
                written_name=written_name
            )
            pers.name=written_name
            pers.first_name = written_name.split(' ')[0]
            temp_work = Work.objects.get(written_name=row['Signatur'])
            pers_work_rel, _ = PersonWork.objects.get_or_create(
                relation_type=pers_work,
                related_person=pers,
                related_work=temp_work
            )

        # add start and end date
            if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                pass
            else:
                date_string_list = row['Datum der Amtshandlung'].split('\n')
                dates = utils.get_start_end(date_string_list)
                if dates:
                    if len(dates) > 1:
                        pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                        pers_work_rel.save()
                    else:
                        pers_work_rel.start_date.written = dates[0]
                        pers_work_rel.save()

        # person-place relation
            pers_pl_rel, _ = PersonPlace.objects.get_or_create(
                relation_type=pers_place,
                related_person=pers,
                related_place=place[0]
            )
            if job_obj:
                pers.profession.add(job_obj)
            pers.save()

In [7]:
# add passive beteiligte Personen
temp_work = None
place = None
parent_pers_work = None
pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="als passiv beteilligte Personen erwähnt", name_reverse='erwähnt als passiv beteilligte Person'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
all_names = None
for df in dfs:
    for i, row in df.iterrows():
        all_names = row['Beteiligte Personen (Erbsinteressenten, Gerhaben, Anweiser, Verkäufer, Verpächter, Käufer, Pächter, ...)'].split('\n')
        temp_work = Work.objects.get(written_name=row['Signatur'])
        for written_name in all_names:
            pers = None
            written_name = written_name.strip()
            pers, _ = Person.objects.get_or_create(
                written_name=written_name
            )
            pers.name=written_name
            pers.first_name = written_name.split(' ')[0]
            pers_work_rel, _ = PersonWork.objects.get_or_create(
                relation_type=pers_work,
                related_person=pers,
                related_work=temp_work
            )

        # add start and end date
            if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                pass
            else:
                date_string_list = row['Datum der Amtshandlung'].split('\n')
                dates = utils.get_start_end(date_string_list)
                if dates:
                    if len(dates) > 1:
                        pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                        pers_work_rel.save()
                    else:
                        pers_work_rel.start_date.written = dates[0]
                        pers_work_rel.save()
            pers.save()

In [8]:
# add aktiv beteiligte Personen
temp_work = None
place = None
parent_pers_work = None
pers_work = None
parent_pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="wird erwähnt", name_reverse='erwähnt'
)
pers_work, _ = PersonWorkRelation.objects.get_or_create(
    name="als aktiv beteilligte Personen erwähnt", name_reverse='erwähnt als aktiv beteilligte Person'
)
pers_work.parent_class = parent_pers_work
pers_work.save()
all_names = None
for df in dfs:
    for i, row in df.iterrows():
        all_names = row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)'].split('\n')
        temp_work = Work.objects.get(written_name=row['Signatur'])
        for written_name in all_names:
            pers = None
            written_name = written_name.strip()
            if written_name.endswith(':'):
                pass
            else:
                pers, _ = Person.objects.get_or_create(
                written_name=written_name
                )
                pers.name=written_name
                pers.first_name = written_name.split(' ')[0]
                pers.save()
                pers_work_rel, _ = PersonWork.objects.get_or_create(
                        relation_type=pers_work,
                        related_person=pers,
                        related_work=temp_work
                )
            
            # add start and end date
                if row['Datum der Amtshandlung'] == 'FALSE' or row['Datum der Amtshandlung'] == 'n. a.':
                    pass
                else:
                    date_string_list = row['Datum der Amtshandlung'].split('\n')
                    dates = utils.get_start_end(date_string_list)
                    if dates:
                        if len(dates) > 1:
                            pers_work_rel.start_date_written, pers_work_rel.end_date_written = dates
                            pers_work_rel.save()
                        else:
                            pers_work_rel.start_date.written = dates[0]
                            pers_work_rel.save()

In [ ]:
# Person.objects.all().delete()

In [ ]:
# Work.objects.all().delete()

In [ ]:
# Place.objects.all().delete()